# Product Neural Network

PNN 是在 DeepCrossing 模型上的修改，在 DeepCrossing 模型中，每一个 field 经过 Embedding 过后直接被拼接在了一起，然后送入全连接层进行后面的计算。在 PNN 模型中，在第一个全连接层之前增加了一个交叉乘积层，交叉的内容是 Embedding 之后的任意两个 field。PNN 的另一个修改是把 DeepCrossing 中的残差 Block 替换成了全连接层。

In [ ]:
# build field dict

import os

BASEDIR = os.getcwd()

fields_dict = {}

with open(BASEDIR + '/assets/datasets/criteo_ctr/small_train.txt') as f:
    line = f.readline()
    line = line.strip('\n')

    while line:
        elems = line.split(' ')

        for i in range(1, len(elems)):
            field, feature, value = elems[i].split(':')
            if field not in fields_dict:
                fields_dict[field] = {}
                fields_dict[field]['field'] = {feature: 0}
                fields_dict[field]['last_idx'] = 0
            elif feature not in fields_dict[field]['field']:
                next_idx = fields_dict[field]['last_idx'] + 1
                fields_dict[field]['field'][feature] = next_idx
                fields_dict[field]['last_idx'] = next_idx
 
        line = f.readline()
        line = line.strip('\n')

for field in fields_dict.keys():
    next_idx = fields_dict[field]['last_idx'] + 1
    fields_dict[field]['field']['none'] = next_idx
    fields_dict[field]['last_idx'] = next_idx

field_cnt = len(fields_dict)
field_cnt

In [ ]:
# build train data

X_train = []
y_train = []
idx = 0
with open(BASEDIR + '/assets/datasets/criteo_ctr/small_train.txt') as f:
    line = f.readline()
    line = line.strip('\n')

    while line:
        elems = line.split(' ')
        y_train.append(int(elems[0]))

        X_train.append([])
        for i in range(field_cnt):
            X_train[idx].append([])

        for i in range(1, len(elems)):
            field, feature, value = elems[i].split(':')
            field_idx = int(field)
            X_train[idx][field_idx].append(fields_dict[field]['field'][feature])

        for i in range(field_cnt):
            if len(X_train[idx][i]) == 0:
                X_train[idx][i].append(fields_dict[str(i)]['field']['none'])
       
        line = f.readline()
        line = line.strip('\n')
        idx += 1


In [ ]:
# build embedding layer

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class ProductNeuralNetwork(nn.Module):
    def __init__(self, fields_dict, embedding_size, z_layers, p_layers):
        super(DeepCrossing, self).__init__()

        self.fields_dict = fields_dict
        self.input_embeddings = []
        self.embedding_size = embedding_size
        self.layers = layers

        for j in range(len(self.fields_dict)):
            self.input_embeddings.append(nn.Linear(len(self.fields_dict[str(j)]['field']), self.embedding_size).double())

        self.z_layers = []
        self.p_layers = []

        for i in range(len(self.input_embeddings)):
            self.z_layers.append(nn.Linear(self.embedding_size + 1, z_layers[i])).double())

        for i in range(len(self.input_embeddings)):
            for j in range(i + 1, len(self.input_embeddings)):
                self.p_layers.append(nn.Linear(self.embedding_size * 2, p_layers[]).double())


        fc_layers = []
        fc_layers.append(nn.Linear(self.embedding_size * 2, self.layers[0]).double())

        for i in range(len(self.layers)):
            fc_layers.append(nn.Linear(self.layers[i-1], self.layers[i]). double())

        self.module = nn.Sequential(fc_layers)

    def forward(self, x):
        embedding_input = []
        batch_size = len(x)
        for idx in range(len(self.input_embeddings)):
            fields_size = len(self.fields_dict[str(idx)]['field'])
            fields_idx_input = torch.zeros(fields_size, batch_size, dtype=dtype, device=device)
            for i in range(batch_size):
                for q in x[i][idx]:
                    fields_idx_input[q, i] = 1.0
            t = self.input_embeddings[idx](fields_idx_input.T)
            embedding_input.append(t)

        fc_layer_input = []
        for i in range(len(embedding_input)):
            fc_layer_input.append()

        # stack_input = torch.cat(embedding_input, 1)
        # out = self.res1(stack_input)
        # out = self.res2(out)
        # out = self.res3(out)
        out = F.sigmoid(self.fc(out))

        return out